In [1]:
import logging
import json
import warnings

import pandas as pd 
from amocrm.v2 import tokens

from utils import (Vocabulary, 
                   AmoCRM,
                   responsible_user_processing, 
                   sale_field_changed_processing, 
                   lead_status_changed_processing)

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
warnings.filterwarnings('ignore')

logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

""" 
PATH_SECRETS - путь до файла secrets.json, который имеет вид:

{"client_id": "", # указать client_id
"client_secret": "", # указать client_secret
"subdomain": "crmsum1",
"redirect_url": "https://lightson.amocrm.ru

"""

PATH_SECRETS = 'secrets.json'
with open(PATH_SECRETS, 'r') as f:
    secrets = json.load(f)
    
storage = tokens.FileTokensStorage()
tokens.default_token_manager(
    client_id=secrets['client_id'],
    client_secret=secrets['client_secret'],
    subdomain=secrets['subdomain'],
    redirect_url=secrets['redirect_url'],
    storage=storage
)

### Create asses token (if not created)

In [ ]:
tokens.default_token_manager.init(code=secrets['code'], skip_error=False)

### Basic Tools Initialization

In [2]:
lead_id = 37903589
# 37925381
vocab = Vocabulary(token_manager=tokens.default_token_manager, secrets=secrets)
amocrm = AmoCRM(token_manager=tokens.default_token_manager, secrets=secrets)
 

### Get all deal events and prepare them for GPT

In [4]:
event_all = amocrm.get_all_lead_info(lead_id=lead_id)
event_all

,type,entity_id,created_by,created_at,specific_data,client,company,sale,lead_status,pipline,responsible
0,initial,37903589,None,2023-09-14 09:36:26,"{'sale': 4404551, 'responsible_user_id': 10076...",NaN,NaN,NaN,Первичный контакт,Воронка,bulatovbogdann@gmail.com
1,sale_field_changed,37903589,None,2023-09-14 09:36:26,{'after': [{'sale_field_value': {'sale': 44041...,NaN,NaN,44041.0,Первичный контакт,Воронка,bulatovbogdann@gmail.com
2,name_field_changed,37903589,None,2023-09-14 09:36:26,{'after': [{'name_field_value': {'name': 'Сдел...,NaN,NaN,44041.0,Первичный контакт,Воронка,bulatovbogdann@gmail.com
3,lead_added,37903589,None,2023-09-14 09:36:26,"{'after': [], 'before': []}",NaN,NaN,44041.0,Первичный контакт,Воронка,bulatovbogdann@gmail.com
4,entity_linked,37903589,bulatovbogdann@gmail.com,2023-10-06 10:10:54,{'after': [{'link': {'entity': {'type': 'compa...,NaN,ИТТММОМА,44041.0,Первичный контакт,Воронка,bulatovbogdann@gmail.com
5,entity_linked,37903589,bulatovbogdann@gmail.com,2023-10-06 10:10:55,{'after': [{'link': {'entity': {'type': 'conta...,Ванов,ИТТММОМА,44041.0,Первичный контакт,Воронка,bulatovbogdann@gmail.com
6,entity_responsible_changed,37903589,bulatovbogdann@gmail.com,2023-10-29 11:56:31,{'after': [{'responsible_user': {'id': 1531225...,Ванов,ИТТММОМА,44041.0,Первичный контакт,Воронка,mlenzovet@gmail.com
7,lead_status_changed,37903589,bulatovbogdann@gmail.com,2023-10-29 12:31:41,"{'after': [{'lead_status': {'id': 60363378, 'p...",Ванов,ИТТММОМА,44041.0,Переговоры,Воронка,mlenzovet@gmail.com
8,entity_responsible_changed,37903589,bulatovbogdann@gmail.com,2023-10-29 12:49:15,{'after': [{'responsible_user': {'id': 2794642...,Ванов,ИТТММОМА,44041.0,Переговоры,Воронка,Технический Пользователь
9,common_note_added,37903589,bulatovbogdann@gmail.com,2023-10-29 12:49:19,"{'after': [{'note': {'id': 399318489}}], 'befo...",Ванов,ИТТММОМА,44041.0,Переговоры,Воронка,Технический Пользователь
